<a href="https://colab.research.google.com/github/jccrews256/ST-554-Project1-Template/blob/main/Task1/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ST 554 Project 1, Task 1
*By Cass Crews*

*Reviewed by Joy Zhou and Trevor Lillywhite*


## Introduction

To be added.

## Importing and Cleaning Data

In [11]:
# Installing UCI machine learning repository module
!pip install ucimlrepo

# Importing key modules
import pandas as pd
import numpy as np
import math
import ucimlrepo as uci

In [26]:
# Reading in the air quality data
air_quality = uci.fetch_ucirepo(id=360)

# Capturing data only
air_quality_df = air_quality.data.features

air_quality_df.head()

air_quality_df.columns

Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH'],
      dtype='object')

In [29]:
# Creating more informative column names
air_quality_df = air_quality_df.rename(columns = {"CO(GT)": "CO_true",
                                 "PT08.S1(CO)": "CO_sensor",
                                 "NMHC(GT)": "NMHC_true",
                                 "C6H6(GT)": "C6H6_true",
                                 "PT08.S2(NMHC)": "NMHC_sensor",
                                 "NOx(GT)": "NOx_true",
                                 "PT08.S3(NOx)": "NOx_sensor",
                                 "NO2(GT)": "NO2_true",
                                 "PT08.S4(NO2)": "NO2_sensor",
                                 "PT08.S5(O3)": "O3_sensor",
                                 "T": "Temp",
                                 "RH": "Rel_Humid",
                                 "AH": "Abs_Humid"})

air_quality_df.columns

Index(['Date', 'Time', 'CO_true', 'CO_sensor', 'NMHC_true', 'C6H6_true',
       'NMHC_sensor', 'NOx_true', 'NOx_sensor', 'NO2_true', 'NO2_sensor',
       'O3_sensor', 'Temp', 'Rel_Humid', 'Abs_Humid'],
      dtype='object')

Let's extract some information on the data frame, such as the total number of observations, each column's data type, and the number of null values by column.

In [30]:
air_quality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         9357 non-null   object 
 1   Time         9357 non-null   object 
 2   CO_true      9357 non-null   float64
 3   CO_sensor    9357 non-null   int64  
 4   NMHC_true    9357 non-null   int64  
 5   C6H6_true    9357 non-null   float64
 6   NMHC_sensor  9357 non-null   int64  
 7   NOx_true     9357 non-null   int64  
 8   NOx_sensor   9357 non-null   int64  
 9   NO2_true     9357 non-null   int64  
 10  NO2_sensor   9357 non-null   int64  
 11  O3_sensor    9357 non-null   int64  
 12  Temp         9357 non-null   float64
 13  Rel_Humid    9357 non-null   float64
 14  Abs_Humid    9357 non-null   float64
dtypes: float64(5), int64(8), object(2)
memory usage: 1.1+ MB


Note that Python does not find any null values, as the non-null counts for each column are equal to the total number of observations. However, the data documentation indicate missing values are denoted by -200 and, therefore, would not be considered null values by Python. We clearly see this issue when we generate some basic summary statistics for the numeric variables using the `describe()` method.

In [31]:
air_quality_df.describe()

,CO_true,CO_sensor,NMHC_true,C6H6_true,NMHC_sensor,NOx_true,NOx_sensor,NO2_true,NO2_sensor,O3_sensor,Temp,Rel_Humid,Abs_Humid
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


When looking at the `min` for each column, we see that all numeric variables have some observations that are missing values. In fact, when looking at `NMHC_true`, we see that more than 75% of observations are missing the true NMHC level.

Let's extract the total number of missing values for each numeric variable so we have a complete understanding of missing value rates.

In [36]:
# Counting missing values
for v in air_quality_df.describe():
    print(v, " Missing Value Count: ", (air_quality_df[v] == -200).sum(), " (", (100*(air_quality_df[v] == -200).sum()/(len(air_quality_df))).round(2), "%)", sep = "")

CO_true Missing Value Count: 1683 (17.99%)
CO_sensor Missing Value Count: 366 (3.91%)
NMHC_true Missing Value Count: 8443 (90.23%)
C6H6_true Missing Value Count: 366 (3.91%)
NMHC_sensor Missing Value Count: 366 (3.91%)
NOx_true Missing Value Count: 1639 (17.52%)
NOx_sensor Missing Value Count: 366 (3.91%)
NO2_true Missing Value Count: 1642 (17.55%)
NO2_sensor Missing Value Count: 366 (3.91%)
O3_sensor Missing Value Count: 366 (3.91%)
Temp Missing Value Count: 366 (3.91%)
Rel_Humid Missing Value Count: 366 (3.91%)
Abs_Humid Missing Value Count: 366 (3.91%)
